## Import libraries and define helper function

In [4]:
import keras
import numpy as np
from pathlib import Path
from keras.models import Model, load_model
from keras.layers import Input, Lambda, Concatenate

def load_model_wrapper(model_hdf5, name=None):  
    model = load_model(model_hdf5, compile=False)
    if name:
        model.name = name
    return model

## Setup directories

Download the data for the notebooks from the dedicated Zenodo link of the CREsted paper. Then use it below.

In [ ]:
data_dir = Path("../../../crested_data/Figure_4/chrombpnet/")

## Load ChromBPnet models and wrap them into one Keras model (GBM + MM lines + Encode DPCLs)

In [9]:
freeze = False

# Step 1: Load separate ChromBPNet models
nb_model_path=f'{data_dir}/og_models/a172_chrombpnet_nobias.h5'
cbp_a172 = load_model_wrapper(nb_model_path, name='a172')
cbp_a172.save(f'{data_dir}/keras_models/chrombpnet_a172.keras')

nb_model_path=f'{data_dir}/og_models/m059j_chrombpnet_nobias.h5'
cbp_m059j = load_model_wrapper(nb_model_path, name='m059j')
cbp_m059j.save(f'{data_dir}/keras_models/chrombpnet_m059j.keras')

nb_model_path=f'{data_dir}/og_models/mm029_chrombpnet_nobias.h5'
cbp_mm029 = load_model_wrapper(nb_model_path, name='mm029')
cbp_mm029.save(f'{data_dir}/keras_models/chrombpnet_mm029.keras')

nb_model_path=f'{data_dir}/og_models/mm099_chrombpnet_nobias.h5'
cbp_mm099 = load_model_wrapper(nb_model_path, name='mm099')
cbp_mm099.save(f'{data_dir}/keras_models/chrombpnet_mm099.keras')

nb_model_path=f'{data_dir}/og_models/ln229_chrombpnet_nobias.h5'
cbp_ln229 = load_model_wrapper(nb_model_path, name='ln229')
cbp_ln229.save(f'{data_dir}/keras_models/chrombpnet_ln229.keras')

nb_model_path=f'{data_dir}/og_models/mm001_chrombpnet_nobias.h5'
cbp_mm001 = load_model_wrapper(nb_model_path, name='mm001')
cbp_mm001.save(f'{data_dir}/keras_models/chrombpnet_mm001.keras')

nb_model_path=f'{data_dir}/og_models/gm12878_chrombpnet_nobias.h5'
cbp_gm12878 = load_model_wrapper(nb_model_path, name='gm12878')
cbp_gm12878.save(f'{data_dir}/keras_models/chrombpnet_gm12878.keras')

nb_model_path=f'{data_dir}/og_models/hepg2_chrombpnet_nobias.h5'
cbp_hepg2 = load_model_wrapper(nb_model_path, name='hepg2')
cbp_hepg2.save(f'{data_dir}/keras_models/chrombpnet_hepg2.keras')

# Step 2: Freeze the layers if needed
if freeze:
    for model in [cbp_a172, cbp_m059j, cbp_mm029, cbp_mm099, cbp_ln229, cbp_mm001, cbp_gm12878, cbp_hepg2]:
        model.trainable = False

# Step 3: Create sub-models for each base model to return only the second (count) output
model1_output2 = Model(inputs=cbp_a172.input, outputs=cbp_a172.output[1])
model2_output2 = Model(inputs=cbp_gm12878.input, outputs=cbp_gm12878.output[1])
model3_output2 = Model(inputs=cbp_hepg2.input, outputs=cbp_hepg2.output[1])
model4_output2 = Model(inputs=cbp_ln229.input, outputs=cbp_ln229.output[1])
model5_output2 = Model(inputs=cbp_m059j.input, outputs=cbp_m059j.output[1])
model6_output2 = Model(inputs=cbp_mm001.input, outputs=cbp_mm001.output[1])
model7_output2 = Model(inputs=cbp_mm029.input, outputs=cbp_mm029.output[1])
model8_output2 = Model(inputs=cbp_mm099.input, outputs=cbp_mm099.output[1])

# Step 4: Define a single input layer and pass through models
input_layer = Input(shape=(2114, 4))

output1 = model1_output2(input_layer)
output2 = model2_output2(input_layer)
output3 = model3_output2(input_layer)
output4 = model4_output2(input_layer)
output5 = model5_output2(input_layer)
output6 = model6_output2(input_layer)
output7 = model7_output2(input_layer)
output8 = model8_output2(input_layer)

# Step 5: Concatenate outputs and wrap them in one model
outputs = Concatenate(axis=-1)([output1, output2, output3, output4, output5, output6, output7, output8])
final_model = Model(inputs=input_layer, outputs=outputs)
print(final_model.summary())

# Step 6: Check output
dummy_input = np.random.rand(1, 2114, 4)
output = final_model.predict(dummy_input)
print(final_model(dummy_input).shape)

# Step 7: Save ensemble model in Keras
final_model.save(f'{data_dir}/chrombpnet_ens.keras')

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 2114, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_4        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_5        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_6        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_7        │ (None, 1)         │  6,339,585 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8)         │          0 │ functional[0][0], │
│ (Concatenate)       │                   │            │ functional_1[0][… │
│                     │                   │            │ functional_2[0][… │
│                     │                   │            │ functional_3[0][… │
│                     │                   │            │ functional_4[0][… │
│                     │                   │            │ functional_5[0][… │
│                     │                   │            │ functional_6[0][… │
│                     │                   │            │ functional_7[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,716,680 (193.47 MB)

 Trainable params: 50,716,680 (193.47 MB)

 Non-trainable params: 0 (0.00 B)

None


I0000 00:00:1743370945.552747  611792 service.cc:152] XLA service 0x14b11c0097b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743370945.577530  611792 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2025-03-30 23:42:27.632809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743370950.061365  611792 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step

I0000 00:00:1743370966.155083  611792 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step
(1, 8)
